---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    import pandas as pd
    university_town =[]
    with open('university_towns.txt') as file:
        for value in file:
            if '[edit]' in value:
                State = value
            else: university_town.append((State,value))
                
                
    towns_df = pd.DataFrame(university_town, columns=['State', 'RegionName'])
    
    region_split = towns_df['RegionName'].str.split('(')
    towns_df['RegionName'] = region_split.str.get(0)
    n_split =  towns_df['RegionName'].str.split('\n')
    towns_df['RegionName'] = n_split.str.get(0)                                             
    state_split = towns_df['State'].str.split('[')                                            
    towns_df['State'] = state_split.str.get(0)
    towns_df.loc[96,'RegionName'] = 'Champaign–Urbana'
    towns_df['State'] = towns_df['State'].str.rstrip()
    towns_df['RegionName'] = towns_df['RegionName'].str.rstrip()
    
    return towns_df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [4]:
def get_recession_start():
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    gdp1 = pd.ExcelFile('gdplev.xls')
    gdp = gdp1.parse(skiprows=219)
    gdp = gdp[['1999q4', 9926.1]]
    
    gdp.columns = ['Quarter','GDP']
    gdp['GDP'] =  gdp['GDP'].apply(pd.to_numeric)
    for i in range(2, len(gdp)):
        if (gdp.iloc[i-2][1] > gdp.iloc[i-1][1]) and (gdp.iloc[i-1][1] > gdp.iloc[i][1]):
            return gdp.iloc[i-2][0]

    
    return gdp.iloc[i-2][0]

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    import pandas as pd
    
    gdp1 = pd.ExcelFile('gdplev.xls')
    gdp = gdp1.parse(skiprows=219)
    gdp = gdp[['1999q4', 9926.1]]
    
    gdp.columns = ['Quarter','GDP']
    gdp['GDP'] =  gdp['GDP'].apply(pd.to_numeric)
    
    
   
    start = get_recession_start()
    start_index = gdp[gdp['Quarter'] == start].index.tolist()[0]
    gdp=gdp.iloc[start_index:]
    for i in range(2, len(gdp)):
        if (gdp.iloc[i-2][1] < gdp.iloc[i-1][1]) and (gdp.iloc[i-1][1] < gdp.iloc[i][1]):
            return gdp.iloc[i][0] 

       
    return gdp.iloc[i][0] 


get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    gdp1 = pd.ExcelFile('gdplev.xls')
    gdp = gdp1.parse(skiprows=219)
    gdp = gdp[['1999q4', 9926.1]]

    
    gdp.columns = ['Quarter','GDP']
    gdp['GDP'] =  gdp['GDP'].apply(pd.to_numeric)
    
    x = gdp.index[gdp['Quarter'] == get_recession_start()][0]

    y = gdp.index[gdp['Quarter'] == get_recession_end()][0]
    
    return gdp.iloc[gdp.iloc[x:y+1, 1 ].idxmin(), 0]

get_recession_bottom()
    
    

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.

    '''
    
    import pandas as pd
    import numpy as np
    df = pd.read_csv('City_Zhvi_AllHomes.csv', header=0)
    
    # Create columns to keep
    cols_to_keep = ['RegionID', 'RegionName', 'State']
    for i in range(2000, 2017):
        for j in range(1, 13):
            if j <= 9:
                if i == 2016 and j == 9:
                    pass
                else:
                    month_str = '0' + str(j)
            else:
                if i == 2016:
                    pass
                else:
                    month_str = str(j)
            cols_to_keep.append(str(i) + '-' + month_str)
    df = df[cols_to_keep]
    
    # Convert two letter state abbreviations to state names
    df['State'] = df['State'].replace(states)

    def convert_to_qtr(ym):
        year, month = ym.split('-')
        if month == '01' or month == '02' or month == '03':
            result = year + 'q1'
        elif month == '04' or month == '05' or month == '06':
            result = year + 'q2'
        elif month == '07' or month == '08' or month == '09':
            result = year + 'q3'
        else:
            result = year + 'q4'
        return result

    # Stack the columns of GDP values and add a quarter column
    df_compiled = df.copy().set_index(['State', 'RegionName', 'RegionID']).stack(dropna=False)
    df_compiled = df_compiled.reset_index().rename(columns={'level_3': 'year_month', 0: 'gdp'})
    df_compiled.drop_duplicates(inplace=True)
    df_compiled['quarter'] = df_compiled['year_month'].apply(convert_to_qtr)
    df_compiled = df_compiled.drop('year_month', axis=1)
    result = df_compiled.pivot_table(values='gdp', index=['State', 'RegionName', 'RegionID'], columns='quarter', aggfunc=np.mean)
    result = result.reset_index()
    result = result.drop('RegionID', axis=1)
    #del result.index.name
    result = result.set_index(['State', 'RegionName'])
    return result


convert_housing_data_to_quarters()

quarter                     2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

quarter                     2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

quarter                     2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

quarter                     2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

quarter                     2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  1094

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    import pandas as pd
    import numpy as np
    from scipy.stats import ttest_ind

    univ_towns = get_list_of_university_towns()
    univ_towns['univ_town'] = True

    # merge the housing data with university town DataFrames
    df = pd.merge(convert_housing_data_to_quarters(), univ_towns, how='outer', left_index=True, right_on=['State', 'RegionName'])
    df['univ_town'] = df['univ_town'].replace({np.NaN: False})

    # Get the recession quarters
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()

    # Parse the year and quarter of the recession quarters
    year_recession_start = int(recession_start[0:4])
    qtr_recession_start = int(recession_start[-1])
    year_recession_bottom = int(recession_bottom[0:4])
    qtr_recession_bottom = int(recession_bottom[-1])

    # get the columns to keep in the merged DataFrame
    cols_to_keep = ['State', 'RegionName', 'univ_town']
    qtrs_to_keep = []
    for i in range(year_recession_start, year_recession_bottom+1):
        for j in range(1, 5):
            if (i == year_recession_start and j < qtr_recession_start)\
                    or (i == year_recession_bottom and j > qtr_recession_bottom):
                pass
            else:
                qtrs_to_keep.append(str(i) + 'q' + str(j))
    df = df[cols_to_keep + qtrs_to_keep]

    # Compute the price_ratio
    df['price_ratio'] = df[recession_bottom] - df[recession_start]

    # t-test to determine if there is a difference between university and non-university towns
    univ_town_price_ratio = df[df['univ_town'] == True]['price_ratio']
    non_univ_town_price_ratio = df[df['univ_town'] == False]['price_ratio']
    st, p = ttest_ind(univ_town_price_ratio, non_univ_town_price_ratio, nan_policy='omit',)

    # get different and better values
    different = False
    if p < 0.01:
        different = True

    # determine which type of town is better
    better = ""
    if univ_town_price_ratio.mean() > non_univ_town_price_ratio.mean():
        better = "university town"
    else:
        better = "non-university town"

    return (different, p, better)

run_ttest()

(True, 0.004325214853459962, 'university town')